Vamos a cargar las librerias y datos necesarios:

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np


uefa_ranking = pd.read_csv('data/uefa_ranking.csv')


# Crear un diccionario para mapear nombres de equipos a números únicos
team_mapping = {team: idx + 1 for idx, team in enumerate(uefa_ranking['Club'])}


#Funcion puntaje para la prediccion de partidos

<h4>Cuartos, primer partido: Atlético Madrid vs Borussia Dortmund

In [3]:
equipo1 = "Atlético Madrid"
equipo2 = "Borussia Dortmund"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


El número de la columna 'Total Points' correspondiente al equipo Atlético Madrid es: 218.0
El número de la columna 'Total Points' correspondiente al equipo Borussia Dortmund es: 179.0


In [4]:
def predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping):
    # Cargar el modelo
    model = tf.keras.models.load_model("model2.h5")

    # Mapear los nombres de los equipos a los números únicos usando team_mapping
    equipo1_map = team_mapping[equipo1]
    equipo2_map = team_mapping[equipo2]

    # Definir la información del partido
    round_number = 9  # Ronda correspondiente

    # Crear una matriz de características para la predicción
    prediction_features = np.array([[round_number, equipo1_map, equipo2_map, total_points_equipo1, total_points_equipo2]])

    # Hacer la predicción utilizando el modelo cargado
    prediction1 = model.predict(prediction_features)

    # Imprimir la predicción
    print(f"Probabilidad de que gane {equipo1} en la ronda 9 es:")
    print(prediction1)

    # Definir la información del partido para la siguiente ronda
    round_number = 10  # Ronda correspondiente

    # Intercambiar los equipos para la siguiente predicción
    equipo1_map, equipo2_map = equipo2_map, equipo1_map
    total_points_equipo1, total_points_equipo2 = total_points_equipo2, total_points_equipo1

    # Crear una matriz de características para la predicción de la siguiente ronda
    prediction_features = np.array([[round_number, equipo1_map, equipo2_map, total_points_equipo1, total_points_equipo2]])

    # Hacer la predicción utilizando el modelo cargado
    prediction2 = model.predict(prediction_features)

    # Imprimir la predicción
    print(f"Probabilidad de que gane {equipo2} en la ronda 10 es:")
    print(prediction2)

    # Calcular el porcentaje de predicción de cada equipo
    percentage_prediction1 = prediction1[0][0]
    percentage_prediction2 = prediction2[0][0]

    # Calcular la probabilidad acumulada de cada equipo
    cumulative_probability1 = (percentage_prediction1 + (1 - percentage_prediction2))/2
    cumulative_probability2 = (percentage_prediction2 + (1 - percentage_prediction1))/2

    # Determinar el ganador
    if cumulative_probability1 > cumulative_probability2:
        winner = equipo1
    else:
        winner = equipo2

    # Retornar el ganador
    return winner

# Predecir el ganador del partido entre "Atlético Madrid" y "Borussia Dortmund"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
Probabilidad de que gane Atlético Madrid en la ronda 9 es:
[[0.7258141]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Probabilidad de que gane Borussia Dortmund en la ronda 10 es:
[[0.22052413]]
El ganador del partido entre Atlético Madrid y Borussia Dortmund es: Atlético Madrid


<h4>Cuartos, segundo partido: Paris Saint-Germain vs FC Barcelona

In [ ]:
equipo1 = "Paris Saint-Germain"
equipo2 = "FC Barcelona"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


In [ ]:
# Predecir el ganador del partido entre "Paris Saint-Germain" y "FC Barcelona"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

<h4>Cuartos, tercer partido: Arsenal vs Bayern München

In [ ]:
equipo1 = "Arsenal"
equipo2 = "Bayern München"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


In [ ]:
# Predecir el ganador del partido entre "Arsenal" y "Bayern München"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

<h4>Cuartos, cuarto partido: Real Madrid vs Manchester City

In [ ]:
equipo1 = "Real Madrid"
equipo2 = "Manchester City"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


In [ ]:
# Predecir el ganador del partido entre "Real Madrid" y "Manchester City"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

<h3>Semis, primer partido: Atlético Madrid vs FC Barcelona (predicción inicial)

In [ ]:
equipo1 = "Atlético Madrid"
equipo2 = "FC Barcelona"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


In [ ]:
# Predecir el ganador del partido entre "Atlético Madrid" y "FC Barcelona"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

<h3>Semis, primer partido: Borussia Dortmund vs Paris Saint-Germain

In [ ]:
equipo1 = "Borussia Dortmund"
equipo2 = "Paris Saint-Germain"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


In [ ]:
# Predecir el ganador del partido entre "Borussia Dortmund" y "Paris Saint-Germain"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

<h3>Semis, segundo partido: Bayern München vs Real Madrid (predicción inicial)

In [ ]:
equipo1 = "Bayern München"
equipo2 = "Real Madrid"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)

In [ ]:
# Predecir el ganador del partido entre "Bayern München" y "Real Madrid"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

<h2>Final: FC Barcelona vs Real Madrid (predicción inicial)

In [ ]:
equipo1 = "FC Barcelona"
equipo2 = "Real Madrid"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)

In [ ]:
# Predecir el ganador del partido entre "FC Barcelona" y "Real Madrid"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

<h1>Por lo tanto, el ganador de la Champions es: REAL MADRID!

<h2>Final: PSG vs Real Madrid (2ª predicción inicial)

In [ ]:
equipo1 = "Paris Saint-Germain"
equipo2 = "Real Madrid"

# Encontrar el índice de la columna 'Total Points'
indice_total_points = uefa_ranking.columns.get_loc('Total Points')

# Obtener el valor de 'Total Points' correspondiente al equipo
total_points_equipo1 = uefa_ranking.loc[uefa_ranking['Club'] == equipo1, 'Total Points'].values[0]
total_points_equipo2 = uefa_ranking.loc[uefa_ranking['Club'] == equipo2, 'Total Points'].values[0]

print("El número de la columna 'Total Points' correspondiente al equipo", equipo1, "es:", total_points_equipo1)
print("El número de la columna 'Total Points' correspondiente al equipo", equipo2, "es:", total_points_equipo2)


In [ ]:
# Predecir el ganador del partido entre "Paris Saint-Germain" y "Real Madrid"
ganador = predecir_ganador(equipo1, equipo2, total_points_equipo1, total_points_equipo2, team_mapping)
print("El ganador del partido entre", equipo1, "y", equipo2, "es:", ganador)

<h1>Por lo tanto, el ganador de la Champions es: REAL MADRID!